# Australia Maximum Temperature Data Viewer

This notebook loads and visualizes the mock meteorological data stored in zarr format.

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd

# Set plot style
plt.style.use('default')
%matplotlib inline

## Load the Dataset

In [ ]:
# Load the zarr dataset
ds = xr.open_zarr('../data/australia_tmax.zarr')
print(ds)

## Dataset Information

In [ ]:
# Display metadata
print("\nDataset Attributes:")
for key, value in ds.attrs.items():
    print(f"  {key}: {value}")

print("\nTemperature Variable Attributes:")
for key, value in ds['tmax'].attrs.items():
    print(f"  {key}: {value}")

## Temperature Statistics

In [ ]:
# Calculate statistics
tmax = ds['tmax'].isel(time=0)  # Get first (and only) time slice

print(f"Temperature Statistics:")
print(f"  Minimum: {float(tmax.min()):.2f}°C")
print(f"  Maximum: {float(tmax.max()):.2f}°C")
print(f"  Mean: {float(tmax.mean()):.2f}°C")
print(f"  Median: {float(tmax.quantile(0.5)):.2f}°C")
print(f"  Std Dev: {float(tmax.std()):.2f}°C")
print(f"\nGrid Dimensions:")
print(f"  Longitude points: {len(ds.lon)}")
print(f"  Latitude points: {len(ds.lat)}")
print(f"  Total grid points: {len(ds.lon) * len(ds.lat):,}")

## Visualize Temperature Map

In [ ]:
# Load Australia boundary
aus_boundary = gpd.read_file('../data/australia_boundary.geojson')

# Create a figure
fig, ax = plt.subplots(figsize=(14, 10))

# Plot the temperature data
im = ax.pcolormesh(
    ds.lon, 
    ds.lat, 
    tmax,
    cmap='RdYlBu_r',  # Red-Yellow-Blue reversed (hot to cold)
    shading='auto',
    vmin=15,
    vmax=45
)

# Plot Australia boundary on top
aus_boundary.boundary.plot(ax=ax, edgecolor='black', linewidth=1.5, label='Australia')

# Add colorbar
cbar = plt.colorbar(im, ax=ax, orientation='vertical', pad=0.02, shrink=0.8)
cbar.set_label('Temperature (°C)', fontsize=12)

# Set labels and title
ax.set_xlabel('Longitude (°E)', fontsize=12)
ax.set_ylabel('Latitude (°N)', fontsize=12)
ax.set_title('Australia - Daily Maximum Temperature\n2024-01-15', fontsize=14, fontweight='bold')

# Set aspect ratio to make it look more geographic
ax.set_aspect('equal')

# Add grid
ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)

# Add legend
ax.legend(loc='upper right')

plt.tight_layout()
plt.show()

## Temperature Distribution Histogram

In [ ]:
# Create histogram
fig, ax = plt.subplots(figsize=(10, 6))

# Calculate median using numpy for compatibility
median_temp = float(np.median(tmax.values))

ax.hist(tmax.values.flatten(), bins=50, color='steelblue', edgecolor='black', alpha=0.7)
ax.axvline(float(tmax.mean()), color='red', linestyle='--', linewidth=2, label=f'Mean: {float(tmax.mean()):.1f}°C')
ax.axvline(median_temp, color='green', linestyle='--', linewidth=2, label=f'Median: {median_temp:.1f}°C')

ax.set_xlabel('Temperature (°C)', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title('Temperature Distribution', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Latitude Profile

Show how temperature varies with latitude (averaged across longitude)

In [ ]:
# Calculate mean temperature for each latitude
lat_profile = tmax.mean(dim='lon')

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(ds.lat, lat_profile, linewidth=2, color='darkred')
ax.fill_between(ds.lat, lat_profile, alpha=0.3, color='red')

ax.set_xlabel('Latitude (°N)', fontsize=12)
ax.set_ylabel('Mean Temperature (°C)', fontsize=12)
ax.set_title('Temperature vs Latitude (Zonal Mean)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Temperature gradient: {float(lat_profile.max() - lat_profile.min()):.1f}°C across latitude range")

## Access Specific Location

Example: Get temperature at a specific coordinate

In [ ]:
# Example coordinates (roughly Sydney area)
example_lat = -33.9
example_lon = 151.2

# Select nearest point
point_temp = ds['tmax'].sel(lat=example_lat, lon=example_lon, method='nearest').isel(time=0)

print(f"Temperature at approximately ({example_lat}°, {example_lon}°):")
print(f"  {float(point_temp):.2f}°C")
print(f"\nActual coordinates:")
print(f"  Lat: {float(point_temp.lat):.2f}°")
print(f"  Lon: {float(point_temp.lon):.2f}°")